In [31]:
from pyClarion import (Agent, Input, Choice, ChunkStore, FixedRules, 
    Family, Atoms, Atom, BaseLevel, Pool, NumDict, Event, Priority)
from datetime import timedelta

import logging
import sys

import numpy as np
import pandas as pd
from typing import *

from utils import RuleWBLA
import math

In [12]:
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(filename)s:%(lineno)d - %(funcName)s - %(message)s')

logger = logging.getLogger("pyClarion.system")
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
# handler.setFormatter(formatter)
logger.addHandler(handler)

In [4]:
class Brick(Atoms):
    horizontal: Atom
    vertical: Atom
    mirror_L: Atom
    half_T: Atom

class GridRows(Atoms):
    r1: Atom
    r2: Atom
    r3: Atom
    r4: Atom
    r5: Atom
    r6: Atom

class GridCols(Atoms):
    c1: Atom
    c2: Atom
    c3: Atom
    c4: Atom
    c5: Atom
    c6: Atom

class QueryRel(Atoms):
    left: Atom
    above: Atom
    right: Atom
    below: Atom

class SignalTokens(Atoms):
    stop_construction: Atom
    continue_construction: Atom

class ConstructionIO(Atoms):
    input_shape1: Atom
    input_shape2: Atom
    input_shape3: Atom
    input_shape4: Atom

    target_shape1: Atom
    target_shape2: Atom
    target_shape3: Atom
    target_shape4: Atom

    input_shape1_row1: Atom
    input_shape1_row2: Atom
    input_shape1_row3: Atom
    input_shape1_col1: Atom
    input_shape1_col2: Atom
    input_shape1_col3: Atom
    
    input_shape2_row1: Atom
    input_shape2_row2: Atom
    input_shape2_row3: Atom
    input_shape2_col1: Atom
    input_shape2_col2: Atom
    input_shape2_col3: Atom
    
    input_shape3_row1: Atom
    input_shape3_row2: Atom
    input_shape3_row3: Atom
    input_shape3_col1: Atom
    input_shape3_col2: Atom
    input_shape3_col3: Atom
    
    input_shape4_row1: Atom
    input_shape4_row2: Atom
    input_shape4_row3: Atom
    input_shape4_col1: Atom
    input_shape4_col2: Atom
    input_shape4_col3: Atom
    
    target_shape1_row1: Atom
    target_shape1_row2: Atom
    target_shape1_row3: Atom
    target_shape1_col1: Atom
    target_shape1_col2: Atom
    target_shape1_col3: Atom

    target_shape2_row1: Atom
    target_shape2_row2: Atom
    target_shape2_row3: Atom
    target_shape2_col1: Atom
    target_shape2_col2: Atom
    target_shape2_col3: Atom

    target_shape3_row1: Atom
    target_shape3_row2: Atom
    target_shape3_row3: Atom
    target_shape3_col1: Atom
    target_shape3_col2: Atom
    target_shape3_col3: Atom
    
    target_shape4_row1: Atom
    target_shape4_row2: Atom
    target_shape4_row3: Atom
    target_shape4_col1: Atom
    target_shape4_col2: Atom
    target_shape4_col3: Atom

    construction_signal: Atom

class ResponseIO(Atoms):
    target_shape1: Atom
    target_shape2: Atom
    target_shape3: Atom
    target_shape4: Atom
    
    target_shape1_row1: Atom
    target_shape1_row2: Atom
    target_shape1_row3: Atom
    target_shape1_col1: Atom
    target_shape1_col2: Atom
    target_shape1_col3: Atom
    
    target_shape2_row1: Atom
    target_shape2_row2: Atom
    target_shape2_row3: Atom
    target_shape2_col1: Atom
    target_shape2_col2: Atom
    target_shape2_col3: Atom
    
    target_shape3_row1: Atom
    target_shape3_row2: Atom
    target_shape3_row3: Atom
    target_shape3_col1: Atom
    target_shape3_col2: Atom
    target_shape3_col3: Atom
    
    target_shape4_row1: Atom
    target_shape4_row2: Atom
    target_shape4_row3: Atom
    target_shape4_col1: Atom
    target_shape4_col2: Atom
    target_shape4_col3: Atom
    
    query_block: Atom
    query_block_reference: Atom
    query_relation: Atom
    output: Atom

class Response(Atoms):
    yes: Atom
    no: Atom

In [5]:
class BrickConstructionTask(Family):
    bricks: Brick
    grid_rows: GridRows
    grid_cols: GridCols
    signal_tokens: SignalTokens
    io: ConstructionIO

class BrickResponseTask(Family):
    bricks: Brick
    grid_rows: GridRows
    grid_cols: GridCols
    query_rel: QueryRel
    io: ResponseIO
    response: Response

In [29]:
class Participant(Agent):  
    construction_space: BrickConstructionTask
    response_space: BrickResponseTask
    construction_input: Input
    response_input: Input
    response_rules: RuleWBLA
    response_blas: BaseLevel
    pool: Pool
    response_choice: Choice
    #TODO: fill in the rest here once ur done below

    def __init__(self, name: str) -> None:
        p = Family() # what is p
        e = Family() # what is e?
        r_construction  = Family() # rule family
        r_response = Family() # rule family 
        c_construction = Family() # construction family
        c_response = Family() # response family
        construction_space = BrickConstructionTask()
        response_space = BrickResponseTask()

        super().__init__(name, p=p, e=e, 
                         construction_space=construction_space, response_space=response_space, 
                         r_construction=r_construction, r_response=r_response, 
                         c_construction=c_construction, c_response=c_response)
        self.construction_space = construction_space
        self.response_space = response_space
        with self:
            self.construction_input = Input("construction_input", (construction_space, construction_space))
            self.response_input = Input("response_input",  (response_space, response_space), reset=False)
            self.response_rules = RuleWBLA("response_rules", p=p, r=r_response, c=c_response, d=response_space, v=response_space, sd=1e-4)
            self.search_space_rules = RuleWBLA("search_space_rules", p=p, r=r_construction, c=c_construction, d=construction_space, v=construction_space, sd=1e-4)
            
            self.response_blas = BaseLevel("blas", p, e, self.response_rules.rules.rules)
            self.search_space_blas = BaseLevel("search_space_blas", p, e, self.search_space_rules.rules.rules)
            self.response_blas.ignore.add(~self.response_rules.rules.rules.nil) 
            self.search_space_blas.ignore.add(~self.search_space_rules.rules.rules.nil)

            self.response_pool = Pool("pool", p, self.response_rules.rules.rules, func=NumDict.sum) # to pool together the blas and condition activations
            self.search_space_pool = Pool("search_space_pool", p, self.search_space_rules.rules.rules, func=NumDict.sum) # similar function

            self.response_choice = Choice("choice", p, (response_space, response_space))
            self.search_space_choice = Choice("search_space_choice", p, (construction_space, construction_space))
        
        self.response_blas.input = self.response_rules.choice.main
        self.search_space_blas.input = self.search_space_rules.choice.main

        self.response_rules.rules.lhs.bu.input = self.response_input.main # TODO: how to do the rules apply a change to the input if i never say to do so . 
        self.search_space_rules.rules.lhs.bu.input = self.construction_input.main

        self.search_space_pool["search_spaces_rules.rules"] = (
            self.search_space_rules.rules.main,  
            lambda d: d.shift(x=1).scale(x=0.5).logit())
        self.response_pool["response_rules.rules"] = (
            self.response_rules.rules.main,
            lambda d: d.shift(x=1).scale(x=0.5).logit())
        
        self.response_pool["blas"] = (
            self.response_blas.main, 
            lambda d: d.bound_min(x=1e-8).log().with_default(c=0.0))
        self.search_space_pool["blas"] = (
            self.search_space_blas.main,
            lambda d: d.bound_min(x=1e-8).log().with_default(c=0.0))
        
        self.response_rules.bla_main = self.response_pool.main #updated site for the response rules to take BLAS into account
        self.search_space_rules.bla_main = self.search_space_pool.main
        #rewire choice input
        self.search_space_rules.choice.input = self.search_space_pool.main
        self.response_rules.choice.input = self.response_pool.main

        self.response_choice.input = self.response_rules.rules.rhs.td.main # bu choice
        self.search_space_choice.input = self.search_space_rules.rules.rhs.td.main

        with self.response_pool.params[0].mutable():
            self.response_pool.params[0][~self.response_pool.p["blas"]] = 2e-1
        with self.search_space_pool.params[0].mutable():
            self.search_space_pool.params[0][~self.search_space_pool.p["blas"]] = 2e-1

    def resolve(self, event: Event) -> None:
        if event.source == self.response_rules.rules.update: # after the rules have updated 
            self.response_blas.update() # timestep update
        elif event.source == self.response_pool.update: # TODO: is this right?
            self.response_rules.trigger()
        elif event.source == self.response_rules.rules.rhs.td.update:
            self.response_choice.trigger() #poll outside the loop in experimental loop
        
        elif event.source == self.search_space_rules.rules.update:
            self.search_space_blas.update()
        elif event.source == self.search_space_pool.update:
            self.search_space_rules.trigger()
        elif event.source == self.search_space_rules.rules.rhs.td.update:
            self.search_space_choice.trigger()
        elif event.source == self.search_space_choice.select:
            #check if indeed we need to stop construction, if triggered the STOP rule
            cur_choice = self.search_space_choice.poll()
            if cur_choice[~self.construction_space.io.construction_signal * ~self.construction_space.signal_tokens] == ~self.construction_space.io.construction_signal * ~self.construction_space.signal_tokens.stop_construction: # TODO: check this
                self.end_construction()
                self.response_input.send(self.search_space_choice.main[0]) # send the choice to the response input -- to make a decision out of 
            else:
                self.construction_input.send(self.search_space_choice.main[0]) # loop it back in --for more selections
        elif event.source == self.end_construction:
            # clear system queue
            self.system.queue.clear() # at this point, you should have a clear target_grid representation built -- correct or incorrect

    def start_construct_trial(self, 
        dt: timedelta, 
        priority: Priority = Priority.PROPAGATION
    ) -> None:
        self.system.schedule(self.start_construct_trial, dt=dt, priority=priority)

    def end_construction(self, 
                         dt: timedelta = timedelta(seconds=0),
                         priority: Priority = Priority.PROPAGATION
                         ) -> None:
        self.system.schedule(self.end_construction, dt=dt, priority=priority)

    def start_response_trial(self, 
        dt: timedelta, 
        priority: Priority = Priority.PROPAGATION
    ) -> None:
        self.system.schedule(self.start_response_trial, dt=dt, priority=priority)
    
    def finish_response_trial(self,
                              dt: timedelta,
        priority: Priority = Priority.PROPAGATION
    ) -> None:
        self.system.schedule(self.finish_response_trial, dt=dt, priority=priority)

    #TODO: providing feedback on constructions
    #TODO:running search for the correct construction
    #TODO: RL for the construction task + rule extraction from results of RL on the bottom level. 

# Knowledge init

In [ ]:
def init_participant_response_rules(participant: Participant) -> None:
    d = participant.response_space
    io = d.io
    bricks = d.bricks
    grid_rows = d.grid_rows
    grid_cols = d.grid_cols
    query_rel = d.query_rel
    response = d.response

    #response rules when given a query and some blocks
    
    # HALF_T X HORIZONTAL
    #i = 0, horizontal is next to the potruding part of the shape, else it is next to the flat part at the bottom
    half_T_left_of_horizontal = [
        (+ io.query_block ** (bricks.half_T if not switcharoo else bricks.horizontal)
        + io.query_block_reference ** (bricks.horizontal if not switcharoo else bricks.half_T)
        + io.query_relation ** (query_rel.left if not switcharoo else query_rel.right)
        + io.target_shape1 ** bricks.half_T
        + io.target_shape4 ** bricks.horizontal
        + io.target_shape1_row1 ** grid_rows[f"r{row}"]
        + io.target_shape1_row2 ** grid_rows[f"r{row}"]
        + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape1_col1 ** grid_cols[f"c{col}"]
        + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape1_col3 ** grid_cols[f"c{col}"]
        + io.target_shape4_row1 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_row2 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_row3 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_col1 ** grid_cols[f"c{col+2-i}"]
        + io.target_shape4_col2 ** grid_cols[f"c{col+3-i}"]
        + io.target_shape4_col3 ** grid_cols[f"c{col+4-i}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(2) for row in range(1, 6) for col in range(1, 3 + i)
    ]

    half_T_right_of_horizontal = [
        (+ io.query_block ** (bricks.half_T if not switcharoo else bricks.horizontal)
        + io.query_block_reference ** (bricks.horizontal if not switcharoo else bricks.half_T)
        + io.query_relation ** (query_rel.right if not switcharoo else query_rel.left)
        + io.target_shape1 ** bricks.half_T
        + io.target_shape4 ** bricks.horizontal
        + io.target_shape1_row1 ** grid_rows[f"r{row}"]
        + io.target_shape1_row2 ** grid_rows[f"r{row}"]
        + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape1_col1 ** grid_cols[f"c{col}"]
        + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape1_col3 ** grid_cols[f"c{col}"]
        + io.target_shape4_row1 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_row2 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_row3 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_col1 ** grid_cols[f"c{col-3}"]
        + io.target_shape4_col2 ** grid_cols[f"c{col-2}"]
        + io.target_shape4_col3 ** grid_cols[f"c{col-1}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(2) for row in range(1, 6) for col in range(4, 6)
    ]
    """
    TODO:
    Check the actual relation for
    1 1
    1 2 2 2 is this ontopness or beside? 
    TODO: Similarly for the other relations involving L like shapes
    """

    half_T_below_horizontal = [
        (+ io.query_block ** (bricks.half_T if not switcharoo else bricks.horizontal)
        + io.query_block_reference ** (bricks.horizontal if not switcharoo else bricks.half_T)
        + io.query_relation ** (query_rel.below if not switcharoo else query_rel.above)
        + io.target_shape1 ** bricks.half_T
        + io.target_shape4 ** bricks.horizontal
        + io.target_shape1_row1 ** grid_rows[f"r{row}"]
        + io.target_shape1_row2 ** grid_rows[f"r{row}"]
        + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape1_col1 ** grid_cols[f"c{col}"]
        + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape1_col3 ** grid_cols[f"c{col}"]
        + io.target_shape4_row1 ** grid_rows[f"r{row-1}"]
        + io.target_shape4_row2 ** grid_rows[f"r{row-1}"]
        + io.target_shape4_row3 ** grid_rows[f"r{row-1}"]
        + io.target_shape4_col1 ** grid_cols[f"c{col-1+i}"]
        + io.target_shape4_col2 ** grid_cols[f"c{col+i}"]
        + io.target_shape4_col3 ** grid_cols[f"c{col+1+i}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(3) for row in range(2, 6) for col in range(1 + (i == 0), 6-i)
    ]

    half_T_above_horizontal = [
        (+ io.query_block ** (bricks.half_T if not switcharoo else bricks.horizontal)
        + io.query_block_reference ** (bricks.horizontal if not switcharoo else bricks.half_T)
        + io.query_relation ** (query_rel.above if not switcharoo else query_rel.below)
        + io.target_shape1 ** bricks.half_T
        + io.target_shape4 ** bricks.horizontal
        + io.target_shape1_row1 ** grid_rows[f"r{row}"]
        + io.target_shape1_row2 ** grid_rows[f"r{row}"]
        + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape1_col1 ** grid_cols[f"c{col}"]
        + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape1_col3 ** grid_cols[f"c{col}"]
        + io.target_shape4_row1 ** grid_rows[f"r{row+2}"]
        + io.target_shape4_row2 ** grid_rows[f"r{row+2}"]
        + io.target_shape4_row3 ** grid_rows[f"r{row+2}"]
        + io.target_shape4_col1 ** grid_cols[f"c{col-2+i}"]
        + io.target_shape4_col2 ** grid_cols[f"c{col-1+i}"]
        + io.target_shape4_col3 ** grid_cols[f"c{col+i}"]
        >>
        + io.output ** response.yes) for i in range(3) for switcharoo in (True, False) for row in range(1, 5) for col in range(3-i, 6-(i==2))
]

    #HALF_T X VERTICAL

    half_T_left_vertical = [
        (+ io.query_block ** (bricks.half_T if not switcharoo else bricks.vertical)
        + io.query_block_reference ** (bricks.vertical if not switcharoo else bricks.half_T)
        + io.query_relation ** (query_rel.left if not switcharoo else query_rel.right)
        + io.target_shape1 ** bricks.half_T
        + io.target_shape3 ** bricks.vertical
        + io.target_shape1_row1 ** grid_rows[f"r{row}"]
        + io.target_shape1_row2 ** grid_rows[f"r{row}"]
        + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape1_col1 ** grid_cols[f"c{col}"]
        + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape1_col3 ** grid_cols[f"c{col}"]
        + io.target_shape3_row1 ** grid_rows[f"r{row+i}"]
        + io.target_shape3_row2 ** grid_rows[f"r{row+i+1}"]
        + io.target_shape3_row3 ** grid_rows[f"r{row+i+2}"]
        + io.target_shape3_col1 ** grid_cols[f"c{col+2}"]
        + io.target_shape3_col2 ** grid_cols[f"c{col+2}"]
        + io.target_shape3_col3 ** grid_cols[f"c{col+2}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(4) for row in range(1+(i if i > 1 else 0), 4 + (math.ceil(i/2))) for col in range(1, 6-(i > 0))
    ]

    half_T_right_vertical = [
        (+ io.query_block ** (bricks.half_T if not switcharoo else bricks.vertical)
        + io.query_block_reference ** (bricks.vertical if not switcharoo else bricks.half_T)
        + io.query_relation ** (query_rel.right if not switcharoo else query_rel.left)
        + io.target_shape1 ** bricks.half_T
        + io.target_shape3 ** bricks.vertical
        + io.target_shape1_row1 ** grid_rows[f"r{row}"]
        + io.target_shape1_row2 ** grid_rows[f"r{row}"]
        + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape1_col1 ** grid_cols[f"c{col}"]
        + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape1_col3 ** grid_cols[f"c{col}"]
        + io.target_shape3_row1 ** grid_rows[f"r{row-2+i}"]
        + io.target_shape3_row2 ** grid_rows[f"r{row-1+i}"]
        + io.target_shape3_row3 ** grid_rows[f"r{row+i}"]
        + io.target_shape3_col1 ** grid_cols[f"c{col-1}"]
        + io.target_shape3_col2 ** grid_cols[f"c{col-1}"]
        + io.target_shape3_col3 ** grid_cols[f"c{col-1}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(4) for row in range(max(1, 3-i), 6-max(0, i-1)) for col in range(2, 6)
    ]

    half_T_below_vertical = [
        (+ io.query_block ** (bricks.half_T if not switcharoo else bricks.vertical)
        + io.query_block_reference ** (bricks.vertical if not switcharoo else bricks.half_T)
        + io.query_relation ** (query_rel.below if not switcharoo else query_rel.above)
        + io.target_shape1 ** bricks.half_T
        + io.target_shape3 ** bricks.vertical
        + io.target_shape1_row1 ** grid_rows[f"r{row}"]
        + io.target_shape1_row2 ** grid_rows[f"r{row}"]
        + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape1_col1 ** grid_cols[f"c{col}"]
        + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape1_col3 ** grid_cols[f"c{col}"]
        + io.target_shape3_row1 ** grid_rows[f"r{row-3}"]
        + io.target_shape3_row2 ** grid_rows[f"r{row-2}"]
        + io.target_shape3_row3 ** grid_rows[f"r{row-3}"]
        + io.target_shape3_col1 ** grid_cols[f"c{col+i}"]
        + io.target_shape3_col2 ** grid_cols[f"c{col+i}"]
        + io.target_shape3_col3 ** grid_cols[f"c{col+i}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(2) for row in range(4, 6) for col in range(1, 6)
    ]

    half_T_above_vertical = [
        (+ io.query_block ** (bricks.half_T if not switcharoo else bricks.vertical)
        + io.query_block_reference ** (bricks.vertical if not switcharoo else bricks.half_T)
        + io.query_relation ** (query_rel.above if not switcharoo else query_rel.below)
        + io.target_shape1 ** bricks.half_T
        + io.target_shape3 ** bricks.vertical
        + io.target_shape1_row1 ** grid_rows[f"r{row}"]
        + io.target_shape1_row2 ** grid_rows[f"r{row}"]
        + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape1_col1 ** grid_cols[f"c{col}"]
        + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape1_col3 ** grid_cols[f"c{col}"]
        + io.target_shape3_row1 ** grid_rows[f"r{row+1}"]
        + io.target_shape3_row2 ** grid_rows[f"r{row+2}"]
        + io.target_shape3_row3 ** grid_rows[f"r{row+3}"]
        + io.target_shape3_col1 ** grid_cols[f"c{col}"]
        + io.target_shape3_col2 ** grid_cols[f"c{col}"]
        + io.target_shape3_col3 ** grid_cols[f"c{col}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for row in range(1, 3) for col in range(1, 6)
    ]

    #HALF_T X MIRROR_L
    half_T_left_mirror_L = [
        (+ io.query_block ** (bricks.half_T if not switcharoo else bricks.mirror_L)
        + io.query_block_reference ** (bricks.mirror_L if not switcharoo else bricks.half_T)
        + io.query_relation ** (query_rel.left if not switcharoo else query_rel.right)
        + io.target_shape1 ** bricks.half_T
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape1_row1 ** grid_rows[f"r{row}"]
        + io.target_shape1_row2 ** grid_rows[f"r{row}"]
        + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape1_col1 ** grid_cols[f"c{col}"]
        + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape1_col3 ** grid_cols[f"c{col}"]
        + io.target_shape2_row1 ** grid_rows[f"r{row-i}"]
        + io.target_shape2_row2 ** grid_rows[f"r{row+1-i}"]
        + io.target_shape2_row3 ** grid_rows[f"r{row+1-i}"]
        + io.target_shape2_col1 ** grid_cols[f"c{col+2+i}"]
        + io.target_shape2_col2 ** grid_cols[f"c{col+1+i}"]
        + io.target_shape2_col3 ** grid_cols[f"c{col+2+i}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(2) for row in range(1+i, 6) for col in range(1, 5-i)
    ]

    """  
    # 1 1 2
    # 1 2 2

    #       2
    # 1 1 2 2
    # 1
    """
    half_T_right_mirror_L = [
        (+ io.query_block ** (bricks.half_T if not switcharoo else bricks.mirror_L)
        + io.query_block_reference ** (bricks.mirror_L if not switcharoo else bricks.half_T)
        + io.query_relation ** (query_rel.right if not switcharoo else query_rel.left)
        + io.target_shape1 ** bricks.half_T
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape1_row1 ** grid_rows[f"r{row}"]
        + io.target_shape1_row2 ** grid_rows[f"r{row}"]
        + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape1_col1 ** grid_cols[f"c{col}"]
        + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape1_col3 ** grid_cols[f"c{col}"]
        + io.target_shape2_row1 ** grid_rows[f"r{row + (-1 if i== 0 else 1)*(i%2 == 0)}"] # 1 up, no up, 1 down.
        + io.target_shape2_row2 ** grid_rows[f"r{row + i}"]
        + io.target_shape2_row3 ** grid_rows[f"r{row+i}"]
        + io.target_shape2_col1 ** grid_cols[f"c{col-1}"]
        + io.target_shape2_col2 ** grid_cols[f"c{col-2}"]
        + io.target_shape2_col3 ** grid_cols[f"c{col-1}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(3) for row in range(1+(i==0), 6-(i==2)) for col in range(3, 6)
    ]

    half_T_below_mirror_L = [
        (+ io.query_block ** (bricks.half_T if not switcharoo else bricks.mirror_L)
        + io.query_block_reference ** (bricks.mirror_L if not switcharoo else bricks.half_T)
        + io.query_relation ** (query_rel.below if not switcharoo else query_rel.above)
        + io.target_shape1 ** bricks.half_T
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape1_row1 ** grid_rows[f"r{row}"]
        + io.target_shape1_row2 ** grid_rows[f"r{row}"]
        + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape1_col1 ** grid_cols[f"c{col}"]
        + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape1_col3 ** grid_cols[f"c{col}"]
        + io.target_shape2_row1 ** grid_rows[f"r{row-2}"]
        + io.target_shape2_row2 ** grid_rows[f"r{row-1}"]
        + io.target_shape2_row3 ** grid_rows[f"r{row-1}"]
        + io.target_shape2_col1 ** grid_cols[f"c{col+1+i}"]
        + io.target_shape2_col2 ** grid_cols[f"c{col+i}"]
        + io.target_shape2_col3 ** grid_cols[f"c{col+1+i}"]
        >>
        + io.output ** response.yes) for i in range(2) for switcharoo in (True, False) for row in range(3, 6) for col in range(1, 6-i)
    ]

    half_T_above_mirror_L = [
        (+ io.query_block ** (bricks.half_T if not switcharoo else bricks.mirror_L)
        + io.query_block_reference ** (bricks.mirror_L if not switcharoo else bricks.half_T)
        + io.query_relation ** (query_rel.above if not switcharoo else query_rel.below)
        + io.target_shape1 ** bricks.half_T
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape1_row1 ** grid_rows[f"r{row}"]
        + io.target_shape1_row2 ** grid_rows[f"r{row}"]
        + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape1_col1 ** grid_cols[f"c{col}"]
        + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape1_col3 ** grid_cols[f"c{col}"]
        + io.target_shape2_row1 ** grid_rows[f"r{row+2}"]
        + io.target_shape2_row2 ** grid_rows[f"r{row+3}"]
        + io.target_shape2_row3 ** grid_rows[f"r{row+3}"]
        + io.target_shape2_col1 ** grid_cols[f"c{col}"]
        + io.target_shape2_col2 ** grid_cols[f"c{col-1}"]
        + io.target_shape2_col3 ** grid_cols[f"c{col}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for row in range(1, 4) for col in range(2, 6)
    ]

    #MIRROR_L X HORIZONTAL
    mirror_L_left_horizontal  = [
        (+ io.query_block ** (bricks.mirror_L if not switcharoo else bricks.horizontal)
        + io.query_block_reference ** (bricks.horizontal if not switcharoo else bricks.mirror_L)
        + io.query_relation ** (query_rel.left if not switcharoo else query_rel.right)
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape4 ** bricks.horizontal
        + io.target_shape2_row1 ** grid_rows[f"r{row}"]
        + io.target_shape2_row2 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_col1 ** grid_cols[f"c{col}"]
        + io.target_shape2_col2 ** grid_cols[f"c{col-1}"]
        + io.target_shape2_col3 ** grid_cols[f"c{col}"]
        + io.target_shape4_row1 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_row2 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_row3 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_col1 ** grid_cols[f"c{col+1}"]
        + io.target_shape4_col2 ** grid_cols[f"c{col+2}"]
        + io.target_shape4_col3 ** grid_cols[f"c{col+3}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(2) for row in range(1, 6) for col in range(2, 4)
    ]

    mirror_L_right_horizontal = [
        (+ io.query_block ** (bricks.mirror_L if not switcharoo else bricks.horizontal)
        + io.query_block_reference ** (bricks.horizontal if not switcharoo else bricks.mirror_L)
        + io.query_relation ** (query_rel.right if not switcharoo else query_rel.left)
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape4 ** bricks.horizontal
        + io.target_shape2_row1 ** grid_rows[f"r{row}"]
        + io.target_shape2_row2 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_col1 ** grid_cols[f"c{col}"]
        + io.target_shape2_col2 ** grid_cols[f"c{col-1}"]
        + io.target_shape2_col3 ** grid_cols[f"c{col}"]
        + io.target_shape4_row1 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_row2 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_row3 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_col1 ** grid_cols[f"c{col-3-i}"]
        + io.target_shape4_col2 ** grid_cols[f"c{col-2-i}"]
        + io.target_shape4_col3 ** grid_cols[f"c{col-1-i}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(2) for row in range(1, 6) for col in range(4+i, 7)
    ]
    """
    2 2 2 1
        1 1

            1
    2 2 2 1 1
    """

    mirror_L_above_horizontal = [
        (+ io.query_block ** (bricks.mirror_L if not switcharoo else bricks.horizontal)
        + io.query_block_reference ** (bricks.horizontal if not switcharoo else bricks.mirror_L)
        + io.query_relation ** (query_rel.below if not switcharoo else query_rel.above)
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape4 ** bricks.horizontal
        + io.target_shape2_row1 ** grid_rows[f"r{row}"]
        + io.target_shape2_row2 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_col1 ** grid_cols[f"c{col}"]
        + io.target_shape2_col2 ** grid_cols[f"c{col-1}"]
        + io.target_shape2_col3 ** grid_cols[f"c{col}"]
        + io.target_shape4_row1 ** grid_rows[f"r{row+2}"]
        + io.target_shape4_row2 ** grid_rows[f"r{row+2}"]
        + io.target_shape4_row3 ** grid_rows[f"r{row+2}"]
        + io.target_shape4_col1 ** grid_cols[f"c{col-2+i}"]
        + io.target_shape4_col2 ** grid_cols[f"c{col-1+i}"]
        + io.target_shape4_col3 ** grid_cols[f"c{col+i}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(3) for row in range(1, 5) for col in range(2 + (i == 0), 7-i)
    ]
    """
        1
      1 1
    2 2 2
    
      1
    1 1
    2 2 2

    1
    1 1
      2 2 2
    """

    mirror_L_below_horizontal = [
        (+ io.query_block ** (bricks.mirror_L if not switcharoo else bricks.horizontal)
        + io.query_block_reference ** (bricks.horizontal if not switcharoo else bricks.mirror_L)
        + io.query_relation ** (query_rel.below if not switcharoo else query_rel.above)
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape4 ** bricks.horizontal
        + io.target_shape2_row1 ** grid_rows[f"r{row}"]
        + io.target_shape2_row2 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_col1 ** grid_cols[f"c{col}"]
        + io.target_shape2_col2 ** grid_cols[f"c{col-1}"]
        + io.target_shape2_col3 ** grid_cols[f"c{col}"]
        + io.target_shape4_row1 ** grid_rows[f"r{row-1}"]
        + io.target_shape4_row2 ** grid_rows[f"r{row-1}"]
        + io.target_shape4_row3 ** grid_rows[f"r{row-1}"]
        + io.target_shape4_col1 ** grid_cols[f"c{col-2+i}"]
        + io.target_shape4_col2 ** grid_cols[f"c{col-1+i}"]
        + io.target_shape4_col3 ** grid_cols[f"c{col+i}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range (3) for row in range(2, 6) for col in range(3-i, 7-i)
    ]
    """
    2 2 2
        1
      1 1
    """

    #MIRROR_L X VERTICAL
    mirror_L_left_vertical  = [
        (+ io.query_block ** (bricks.mirror_L if not switcharoo else bricks.vertical)
        + io.query_block_reference ** (bricks.vertical if not switcharoo else bricks.mirror_L)
        + io.query_relation ** (query_rel.left if not switcharoo else query_rel.right)
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape3 ** bricks.vertical
        + io.target_shape2_row1 ** grid_rows[f"r{row}"]
        + io.target_shape2_row2 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_col1 ** grid_cols[f"c{col}"]
        + io.target_shape2_col2 ** grid_cols[f"c{col-1}"]
        + io.target_shape2_col3 ** grid_cols[f"c{col}"]
        + io.target_shape3_row1 ** grid_rows[f"r{row-1+i}"]
        + io.target_shape3_row2 ** grid_rows[f"r{row+i}"]
        + io.target_shape3_row3 ** grid_rows[f"r{row+1+i}"]
        + io.target_shape3_col1 ** grid_cols[f"c{col+1}"]
        + io.target_shape3_col2 ** grid_cols[f"c{col+1}"]
        + io.target_shape3_col3 ** grid_cols[f"c{col+1}"]
        >>
        + io.output ** response.yes) 
        for switcharoo in (True, False) for i in range(3) for row in range(1+(i==0), 6-i) for col in range(2, 6)
    ]

    """
    1 2
    1 1 2
        2

    1 
    1 1 2
        2
        2
    """

    mirror_L_right_vertical = [
        (+ io.query_block ** (bricks.mirror_L if not switcharoo else bricks.vertical)
        + io.query_block_reference ** (bricks.vertical if not switcharoo else bricks.mirror_L)
        + io.query_relation ** (query_rel.right if not switcharoo else query_rel.left)
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape3 ** bricks.vertical
        + io.target_shape2_row1 ** grid_rows[f"r{row}"]
        + io.target_shape2_row2 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_col1 ** grid_cols[f"c{col}"]
        + io.target_shape2_col2 ** grid_cols[f"c{col-1}"]
        + io.target_shape2_col3 ** grid_cols[f"c{col}"]
        + io.target_shape3_row1 ** grid_rows[f"r{row+1-i}"]
        + io.target_shape3_row2 ** grid_rows[f"r{row+2-i}"]
        + io.target_shape3_row3 ** grid_rows[f"r{row+3-i}"]
        + io.target_shape3_col1 ** grid_cols[f"c{col-2 + (i == 3)}"]
        + io.target_shape3_col2 ** grid_cols[f"c{col-2 + (i == 3)}"]
        + io.target_shape3_col3 ** grid_cols[f"c{col-2 + (i == 3)}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(4) for row in range(1+min(0, i-1), min(4+i, 6)) for col in range(3 - (i == 3), 7)
    ]

    """
    2   
    2   1
    2 1 1

    2   1
    2 1 1
    2

        1
    2 1 1
    2
    2

    2
    2
    2 1
    1 1
    """

    mirror_L_above_vertical = [
        (+ io.query_block ** (bricks.mirror_L if not switcharoo else bricks.vertical)
        + io.query_block_reference ** (bricks.vertical if not switcharoo else bricks.mirror_L)
        + io.query_relation ** (query_rel.above if not switcharoo else query_rel.below)
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape3 ** bricks.vertical
        + io.target_shape2_row1 ** grid_rows[f"r{row}"]
        + io.target_shape2_row2 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_col1 ** grid_cols[f"c{col}"]
        + io.target_shape2_col2 ** grid_cols[f"c{col-1}"]
        + io.target_shape2_col3 ** grid_cols[f"c{col}"]
        + io.target_shape3_row1 ** grid_rows[f"r{row+2}"]
        + io.target_shape3_row2 ** grid_rows[f"r{row+3}"]
        + io.target_shape3_row3 ** grid_rows[f"r{row+4}"]
        + io.target_shape3_col1 ** grid_cols[f"c{col-1+i}"]
        + io.target_shape3_col2 ** grid_cols[f"c{col-1+i}"]
        + io.target_shape3_col3 ** grid_cols[f"c{col-1+i}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(2) for row in range(1, 3) for col in range(2, 7)
    ]

    """
    1
    1 1
    2
    2
    2 

    1
    1 1
      2
      2
      2
    """

    mirror_L_below_vertical = [
        (+ io.query_block ** (bricks.mirror_L if not switcharoo else bricks.vertical)
        + io.query_block_reference ** (bricks.vertical if not switcharoo else bricks.mirror_L)
        + io.query_relation ** (query_rel.below if not switcharoo else query_rel.above)
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape3 ** bricks.vertical
        + io.target_shape2_row1 ** grid_rows[f"r{row}"]
        + io.target_shape2_row2 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_row3 ** grid_rows[f"r{row+1}"]
        + io.target_shape2_col1 ** grid_cols[f"c{col}"]
        + io.target_shape2_col2 ** grid_cols[f"c{col-1}"]
        + io.target_shape2_col3 ** grid_cols[f"c{col}"]
        + io.target_shape3_row1 ** grid_rows[f"r{row-3}"]
        + io.target_shape3_row2 ** grid_rows[f"r{row-2}"]
        + io.target_shape3_row3 ** grid_rows[f"r{row-1}"]
        + io.target_shape3_col1 ** grid_cols[f"c{col}"]
        + io.target_shape3_col2 ** grid_cols[f"c{col}"]
        + io.target_shape3_col3 ** grid_cols[f"c{col}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for row in range(4, 6) for col in range(2, 7)
    ]

    """
      2
      2
      2
      1
    1 1
    """

    #HORIZONTAL X VERTICAL
    horizontal_left_vertical  = [
        (+ io.query_block ** (bricks.horizontal if not switcharoo else bricks.vertical)
        + io.query_block_reference ** (bricks.vertical if not switcharoo else bricks.horizontal)
        + io.query_relation ** (query_rel.left if not switcharoo else query_rel.right)
        + io.target_shape3 ** bricks.horizontal
        + io.target_shape4 ** bricks.vertical
        + io.target_shape3_row1 ** grid_rows[f"r{row}"]
        + io.target_shape3_row2 ** grid_rows[f"r{row}"]
        + io.target_shape3_row3 ** grid_rows[f"r{row}"]
        + io.target_shape3_col1 ** grid_cols[f"c{col}"]
        + io.target_shape3_col2 ** grid_cols[f"c{col+1}"] 
        + io.target_shape3_col3 ** grid_cols[f"c{col+2}"]
        + io.target_shape4_row1 ** grid_rows[f"r{row-2+i}"]
        + io.target_shape4_row2 ** grid_rows[f"r{row-1+i}"]
        + io.target_shape4_row3 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_col1 ** grid_cols[f"c{col+3}"]
        + io.target_shape4_col2 ** grid_cols[f"c{col+3}"]
        + io.target_shape4_col3 ** grid_cols[f"c{col+3}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(3) for row in range(3-i, 7-i) for col in range(1, 4)
    ]

    """
        2
        2
    1 1 1 2

        2
    1 1 1 2
        2

    1 1 1 2
        2
        2      
    """

    horizontal_right_vertical = [
        (+ io.query_block ** (bricks.horizontal if not switcharoo else bricks.vertical)
        + io.query_block_reference ** (bricks.vertical if not switcharoo else bricks.horizontal)
        + io.query_relation ** (query_rel.right if not switcharoo else query_rel.left)
        + io.target_shape3 ** bricks.horizontal
        + io.target_shape4 ** bricks.vertical
        + io.target_shape3_row1 ** grid_rows[f"r{row}"]
        + io.target_shape3_row2 ** grid_rows[f"r{row}"]
        + io.target_shape3_row3 ** grid_rows[f"r{row}"]
        + io.target_shape3_col1 ** grid_cols[f"c{col}"]
        + io.target_shape3_col2 ** grid_cols[f"c{col+1}"] 
        + io.target_shape3_col3 ** grid_cols[f"c{col+2}"]
        + io.target_shape4_row1 ** grid_rows[f"r{row-2+i}"]
        + io.target_shape4_row2 ** grid_rows[f"r{row-1+i}"]
        + io.target_shape4_row3 ** grid_rows[f"r{row+i}"]
        + io.target_shape4_col1 ** grid_cols[f"c{col-1}"]
        + io.target_shape4_col2 ** grid_cols[f"c{col-1}"]
        + io.target_shape4_col3 ** grid_cols[f"c{col-1}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(3) for row in range(3-i, 7-i) for col in range(2, 5)
    ]

    """
    2
    2
    2 1 1 1

    2
    2 1 1 1
    2

    2
    2
    2 1 1 1
    """

    horizontal_above_vertical = [
        (+ io.query_block ** (bricks.horizontal if not switcharoo else bricks.vertical)
        + io.query_block_reference ** (bricks.vertical if not switcharoo else bricks.horizontal)
        + io.query_relation ** (query_rel.above if not switcharoo else query_rel.below)
        + io.target_shape3 ** bricks.horizontal
        + io.target_shape4 ** bricks.vertical
        + io.target_shape3_row1 ** grid_rows[f"r{row}"]
        + io.target_shape3_row2 ** grid_rows[f"r{row}"]
        + io.target_shape3_row3 ** grid_rows[f"r{row}"]
        + io.target_shape3_col1 ** grid_cols[f"c{col}"]
        + io.target_shape3_col2 ** grid_cols[f"c{col+1}"] 
        + io.target_shape3_col3 ** grid_cols[f"c{col+2}"]
        + io.target_shape4_row1 ** grid_rows[f"r{row+1}"]
        + io.target_shape4_row2 ** grid_rows[f"r{row+2}"]
        + io.target_shape4_row3 ** grid_rows[f"r{row+3}"]
        + io.target_shape4_col1 ** grid_cols[f"c{col+i}"]
        + io.target_shape4_col2 ** grid_cols[f"c{col+i}"]
        + io.target_shape4_col3 ** grid_cols[f"c{col+1+i}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(3) for row in range(1, 4) for col in range(1, 5)
    ]

    """
    1 1 1
    2
    2
    2

    1 1 1
    2
    2
    2

    1 1 1
        2
        2
        2
    """

    horizontal_below_vertical = [
        (+ io.query_block ** (bricks.horizontal if not switcharoo else bricks.vertical)
        + io.query_block_reference ** (bricks.vertical if not switcharoo else bricks.horizontal)
        + io.query_relation ** (query_rel.below if not switcharoo else query_rel.left)
        + io.target_shape3 ** bricks.horizontal
        + io.target_shape4 ** bricks.vertical
        + io.target_shape3_row1 ** grid_rows[f"r{row}"]
        + io.target_shape3_row2 ** grid_rows[f"r{row}"]
        + io.target_shape3_row3 ** grid_rows[f"r{row}"]
        + io.target_shape3_col1 ** grid_cols[f"c{col}"]
        + io.target_shape3_col2 ** grid_cols[f"c{col+1}"] 
        + io.target_shape3_col3 ** grid_cols[f"c{col+2}"]
        + io.target_shape4_row1 ** grid_rows[f"r{row-3}"]
        + io.target_shape4_row2 ** grid_rows[f"r{row-2}"]
        + io.target_shape4_row3 ** grid_rows[f"r{row-1}"]
        + io.target_shape4_col1 ** grid_cols[f"c{col+i}"]
        + io.target_shape4_col2 ** grid_cols[f"c{col+i}"]
        + io.target_shape4_col3 ** grid_cols[f"c{col+i}"]
        >>
        + io.output ** response.yes) for switcharoo in (True, False) for i in range(3) for row in range(4, 7) for col in range(1, 5)
    ]
    """
    2
    2
    2
    1 1 1

    2
    2
    2
    1 1 1

        2
        2
        2
    1 1 1
    """

    #no rule
    no_response_rule = [
        (+ io.query_block ** q_block
        + io.query_block_reference ** q_block_ref
        + io.query_relation ** rel
        >>
        + io.output ** response.no)

        for q_block in (bricks.half_T, bricks.mirror_L, bricks.horizontal, bricks.vertical)
          for q_block_ref in (bricks.half_T, bricks.mirror_L, bricks.horizontal, bricks.vertical)
            for rel in (query_rel.left, query_rel.right, query_rel.above, query_rel.below) if q_block != q_block_ref
        
    ]

    participant.response_rules.rules.compile(*(half_T_left_of_horizontal + half_T_left_vertical + half_T_left_mirror_L + 
                                      half_T_right_of_horizontal + half_T_right_vertical + half_T_right_mirror_L +
                                      half_T_below_horizontal + half_T_below_vertical + half_T_below_mirror_L +
                                      half_T_above_horizontal + half_T_above_vertical + half_T_above_mirror_L +
                                      mirror_L_left_horizontal + mirror_L_left_vertical +
                                      mirror_L_right_horizontal + mirror_L_right_vertical +
                                      mirror_L_below_horizontal + mirror_L_below_vertical +
                                      mirror_L_above_horizontal + mirror_L_above_vertical +
                                      horizontal_left_vertical +
                                      horizontal_right_vertical +
                                      horizontal_above_vertical +
                                      horizontal_below_vertical + no_response_rule))
    

In [ ]:
def init_participant_construction_rules(participant: Participant) -> None:
    d = participant.construction_space
    io = d.io
    bricks = d.bricks
    con_signal = d.signal_tokens
    grid_rows = d.grid_rows
    grid_cols = d.grid_cols

    #FIRST PLACEMENT RULES
    """
    1 1
    1
    """
    half_T_first_placement_rule = [
        (   + io.input_shape1 ** bricks.half_T
            + io.input_shape1_row1 ** grid_rows[f"r{row}"]
            + io.input_shape1_row2 ** grid_rows[f"r{row}"]
            + io.input_shape1_row3 ** grid_rows[f"r{row+1}"]
            + io.input_shape1_col1 ** grid_cols[f"c{col}"]
            + io.input_shape1_col2 ** grid_cols[f"c{col+1}"]
            + io.input_shape1_col3 ** grid_cols[f"c{col}"]

            >>
            + io.target_shape1 ** bricks.half_T
            + io.target_shape1_row1 ** grid_rows[f"r{row}"]
            + io.target_shape1_row2 ** grid_rows[f"r{row}"]
            + io.target_shape1_row3 ** grid_rows[f"r{row+1}"]
            + io.target_shape1_col1 ** grid_cols[f"c{col}"]
            + io.target_shape1_col2 ** grid_cols[f"c{col+1}"]
            + io.target_shape1_col3 ** grid_cols[f"c{col}"]
         )
         for row in range(1, 6) for col in range(1, 6)
    ]

    mirror_L_first_placement_rule = [
        ( + io.input_shape2 ** bricks.mirror_L
          + io.input_shape2_row1 ** grid_rows[f"r{row}"]
            + io.input_shape2_row2 ** grid_rows[f"r{row+1}"]
            + io.input_shape2_row3 ** grid_rows[f"r{row+1}"]
            + io.input_shape2_col1 ** grid_cols[f"c{col}"]
            + io.input_shape2_col2 ** grid_cols[f"c{col-1}"]
            + io.input_shape2_col3 ** grid_cols[f"c{col}"]
            >>
            + io.target_shape2 ** bricks.mirror_L

            + io.target_shape2_row1 ** grid_rows[f"r{row}"]
            + io.target_shape2_row2 ** grid_rows[f"r{row+1}"]
            + io.target_shape2_row3 ** grid_rows[f"r{row+1}"]
            + io.target_shape2_col1 ** grid_cols[f"c{col}"]
            + io.target_shape2_col2 ** grid_cols[f"c{col-1}"]
            + io.target_shape2_col3 ** grid_cols[f"c{col}"]  
        )
        for row in range(1, 6) for col in range(2, 7)
    ]
    """
      1
    1 1
    """

    horizontal_first_placement_rule = [
        ( + io.input_shape3 ** bricks.horizontal
            + io.input_shape3_row1 ** grid_rows[f"r{row}"]
                + io.input_shape3_row2 ** grid_rows[f"r{row}"]
                + io.input_shape3_row3 ** grid_rows[f"r{row}"]
                + io.input_shape3_col1 ** grid_cols[f"c{col}"]
                + io.input_shape3_col2 ** grid_cols[f"c{col+1}"]
                + io.input_shape3_col3 ** grid_cols[f"c{col+2}"]
                >>
                + io.target_shape3 ** bricks.horizontal
    
                + io.target_shape3_row1 ** grid_rows[f"r{row}"]
                + io.target_shape3_row2 ** grid_rows[f"r{row}"]
                + io.target_shape3_row3 ** grid_rows[f"r{row}"]
                + io.target_shape3_col1 ** grid_cols[f"c{col}"]
                + io.target_shape3_col2 ** grid_cols[f"c{col+1}"]
                + io.target_shape3_col3 ** grid_cols[f"c{col+2}"]  
            )
        for row in range(1, 7) for col in range(1, 5)
    ]

    vertical_first_placement_rule = [
        ( + io.input_shape4 ** bricks.vertical
            + io.input_shape4_row1 ** grid_rows[f"r{row}"]
                + io.input_shape4_row2 ** grid_rows[f"r{row+1}"]
                + io.input_shape4_row3 ** grid_rows[f"r{row+2}"]
                + io.input_shape4_col1 ** grid_cols[f"c{col}"]
                + io.input_shape4_col2 ** grid_cols[f"c{col}"]
                + io.input_shape4_col3 ** grid_cols[f"c{col}"]
                >>
                + io.target_shape4 ** bricks.vertical

                + io.target_shape4_row1 ** grid_rows[f"r{row}"]
                + io.target_shape4_row2 ** grid_rows[f"r{row+1}"]
                + io.target_shape4_row3 ** grid_rows[f"r{row+2}"]
                + io.target_shape4_col1 ** grid_cols[f"c{col}"]
                + io.target_shape4_col2 ** grid_cols[f"c{col}"]
                + io.target_shape4_col3 ** grid_cols[f"c{col}"]  
            )
        for row in range(1, 5) for col in range(1, 7)
    ]

    #SUBSEQUENCE PLACEMENT RULES

    # END_CONSTRUCTION RULE
    # if all four blocks have been used, then stop construction
    #TODO: is it possible for a dimension to not be bound with anything? that is the empty state yes? -- 0.0 activation value yea?
    stop_construction_rule = [(
        + io.target_shape1 ** bricks.half_T
        + io.target_shape2 ** bricks.mirror_L
        + io.target_shape3 ** bricks.vertical
        + io.target_shape4 ** bricks.horizontal

        >>
        + io.construction_signal ** con_signal.stop_construction)
    ]

    # participant.search_space_rules.rules.compile(
    #     *(stop_construction_rule + half_T_first_placement_rule + mirror_L_first_placement_rule + horizontal_first_placement_rule + vertical_first_placement_rule)
    # )

    # temporary: to evaluate the functioning of response rules
    dummy_stop_construction_rule = [(
        #TODO: empty condition?
        + io[f"input_shape{shape_no}"] ** bricks[brick_name]
        >>
        io.construction_signal ** con_signal.stop_construction 
    )
     for (shape_no, brick_name) in zip(range(1, 5), ["half_T", "mirror_L", "vertical", "horizontal"])
    ]

    participant.search_space_rules.rules.compile(
        *dummy_stop_construction_rule
    )

In [19]:
def present_stimulus(d:BrickConstructionTask, stim_grid: np.ndarray):
    stim_bricks = np.unique(stim_grid)
    stim_bricks = stim_bricks[stim_bricks != 0]

    brick_map = {1: d.bricks.half_T, 2: d.bricks.mirror_L, 3: d.bricks.vertical, 4: d.bricks.horizontal}
    row_map = {1: d.grid_rows.r1, 2: d.grid_rows.r2, 3: d.grid_rows.r3, 4: d.grid_rows.r4, 5: d.grid_rows.r5, 6: d.grid_rows.r6}
    col_map = {1: d.grid_cols.c1, 2: d.grid_cols.c2, 3: d.grid_cols.c3, 4: d.grid_cols.c4, 5: d.grid_cols.c5, 6: d.grid_cols.c6}
    
    brick_row_map = {1: {1: d.io.input_shape1_row1, 2: d.io.input_shape1_row2, 3: d.io.input_shape1_row3}, 2: {1: d.io.input_shape2_row1, 2: d.io.input_shape2_row2, 3: d.io.input_shape2_row3}, 3: {1: d.io.input_shape3_row1, 2: d.io.input_shape3_row2, 3: d.io.input_shape3_row3}, 4: {1: d.io.input_shape4_row1, 2: d.io.input_shape4_row2, 3: d.io.input_shape4_row3}}
    brick_col_map = {1: {1: d.io.input_shape1_col1, 2: d.io.input_shape1_col2, 3: d.io.input_shape1_col3}, 2: {1: d.io.input_shape2_col1, 2: d.io.input_shape2_col2, 3: d.io.input_shape2_col3}, 3: {1: d.io.input_shape3_col1, 2: d.io.input_shape3_col2, 3: d.io.input_shape3_col3}, 4: {1: d.io.input_shape4_col1, 2: d.io.input_shape4_col2, 3: d.io.input_shape4_col3}}
    shape_brick_map = {1: d.io.input_shape1, 2: d.io.input_shape2, 3: d.io.input_shape3, 4: d.io.input_shape4}
    
    in_send_val = None
    for i, brick in enumerate(stim_bricks):
        # brick row indices: 
        row_indices = np.where(stim_grid == brick)[0] + 1
        col_indices = np.where(stim_grid == brick)[1] + 1

        if not in_send_val:
            in_send_val = (+ shape_brick_map[brick] ** brick_map[brick] 
                           + brick_row_map[brick][1] ** row_map[row_indices[0]] 
                           + brick_row_map[brick][2] **  row_map[row_indices[1]]
                           + brick_row_map[brick][3] ** row_map[row_indices[2]]
                           + brick_col_map[brick][1] ** col_map[col_indices[0]]
                           + brick_col_map[brick][2] ** col_map[col_indices[1]]
                           + brick_col_map[brick][3] ** col_map[col_indices[2]])
        else:
            in_send_val = (in_send_val
                            + shape_brick_map[brick] ** brick_map[brick]
                            + brick_row_map[brick][1] ** row_map[row_indices[0]]
                            + brick_row_map[brick][2] ** row_map[row_indices[1]]
                            + brick_row_map[brick][3] ** row_map[row_indices[2]]
                            + brick_col_map[brick][1] ** col_map[col_indices[0]]
                            + brick_col_map[brick][2] ** col_map[col_indices[1]]
                            + brick_col_map[brick][3] ** col_map[col_indices[2]])
            
    return in_send_val

def load_trial(construction_space: BrickConstructionTask, response_space: BrickResponseTask ,trial, t_type="test", q_type="query"):
    d = response_space
    
    grid_name = trial["Grid_Name"]
    
    stim_grid = np.load(f"/Users/mishaal/personalproj/clarion_replay/processed/{t_type}_data/{t_type}_stims/{grid_name}.npy")         
    chunk_grid = present_stimulus(construction_space, stim_grid)
    
    query_map = {1: d.query_rel.left, 2: d.query_rel.above, 3: d.query_rel.right, 4: d.query_rel.below}
    brick_map = {1: d.bricks.half_T, 2: d.bricks.mirror_L, 3: d.bricks.vertical, 4: d.bricks.horizontal}

    if t_type == "test":
        chunk_test = ( + d.io.query_relation ** query_map[trial["Q_Relation"]] 
                      + d.io.query_block ** brick_map[trial["Q_Brick_Left"]]
                      + d.io.query_block_reference ** brick_map[trial["Q_Brick_Right"]])
    elif t_type == "train" and q_type == "query":
        # choose 2 blocks randomly
        blocks = np.random.choice((t := np.unique(stim_grid))[t != 0], 2, replace=False)
        # choose a relation randomly
        relation = np.random.choice([1, 2, 3, 4], 1)
        chunk_test = ( + d.io.query_relation ** query_map[relation[0]] 
                      + d.io.query_block ** brick_map[blocks[0]]
                      + d.io.query_block_reference ** brick_map[blocks[1]])
    else: chunk_test = ()

    print("Stimulus grid: ", stim_grid)
    if q_type == "query" and t_type == "test":
        print("Query brick 1: ", trial["Q_Brick_Left"])
        print("Query brick 2: ", trial["Q_Brick_Right"])
        print("Query relation: ", trial["Q_Relation"])
    elif q_type == "query":
        print("Query brick 1: ", blocks[0])
        print("Query brick 2: ", blocks[1])
        print("Query relation: ", relation[0])

    return chunk_grid, chunk_test

# Simulation

In [ ]:
def run_participant_session(participant: Participant, session_df: pd.DataFrame, session_type="train", q_type="query"):
    global rule_defs
    results = []
    trials = []
    # Knowledge initialization
    init_participant_response_rules(participant)
    init_participant_construction_rules(participant)
    
    for _, trial in session_df.iterrows():
        trials.append(trial)
        break # testing
    
    participant.start_construct_trial(timedelta(seconds=1))
    while participant.system.queue:
        event = participant.system.advance()
        if event.source == participant.start_construct_trial:
            if not trials: break
            #load the next trial
            trial = trials.pop(0)
            grid_stimulus, test_query = load_trial(participant.construction_space, participant.response_space, trial, t_type=session_type, q_type=q_type)
            participant.construction_input.send(grid_stimulus) # TODO: have a timeout somehow: but how to do timeout wihout proper timinmg constraints for the various events in the queue?
        elif event.source == participant.end_construction:
            participant.start_response_trial(timedelta()) #TODO: checkout the actual time delays
        elif event.source == participant.start_response_trial:
            participant.response_input.send(test_query)
        elif event.source == participant.response_rules.rules.rhs.td.update:
            participant.response_choice.select()
        elif event.source == participant.response_choice.select:
            results.append((event.time, participant.response_choice.poll())) #TODO: come up with a way to save the sequences of search space rules that were activates -- is there in sample attribute of the choice in a rule i believe?
            participant.finish_response_trial(timedelta())
        elif event.source == participant.finish_response_trial:
            participant.start_construct_trial(timedelta())

In [30]:
trials_df = pd.read_csv("~/personalproj/clarion_replay/processed/test_data/all_test_data.csv")
run_participant_session(Participant("p1"), trials_df)

event 0x0000 00:00:00.00 096 0 response_rules.rules.compile
    Added the following new rule(s)
    rule r_response:response_rules.rules:_0
        chunk c_response:response_rules.rules.lhs:_0
            + io.query_block ** bricks.horizontal
            + io.query_block_reference ** bricks.half_T
            + io.query_relation ** query_rel.right
            + io.target_shape1 ** bricks.half_T
            + io.target_shape4 ** bricks.horizontal
            + io.target_shape1_row1 ** grid_rows.r1
            + io.target_shape1_row2 ** grid_rows.r1
            + io.target_shape1_row3 ** grid_rows.r2
            + io.target_shape1_col1 ** grid_cols.c4
            + io.target_shape1_col2 ** grid_cols.c5
            + io.target_shape1_col3 ** grid_cols.c4
            + io.target_shape4_row1 ** grid_rows.r1
            + io.target_shape4_row2 ** grid_rows.r1
            + io.target_shape4_row3 ** grid_rows.r1
            + io.target_shape4_col1 ** grid_cols.c1
            + io.target_shape4

19558it [00:00, 367705.69it/s]
1190it [00:00, 267353.46it/s]

event 0x0000 00:00:00.00 096 1 search_space_rules.rules.compile
    Added the following new rule(s)
    rule r_construction:search_space_rules.rules:_0
        chunk c_construction:search_space_rules.rules.lhs:_0
            + io.input_shape1 ** bricks.half_T
        >>
        chunk c_construction:search_space_rules.rules.rhs:_0
            + io.construction_signal ** signal_tokens.stop_construction
    rule r_construction:search_space_rules.rules:_1
        chunk c_construction:search_space_rules.rules.lhs:_1
            + io.input_shape2 ** bricks.mirror_L
        >>
        chunk c_construction:search_space_rules.rules.rhs:_1
            + io.construction_signal ** signal_tokens.stop_construction
    rule r_construction:search_space_rules.rules:_2
        chunk c_construction:search_space_rules.rules.lhs:_2
            + io.input_shape3 ** bricks.vertical
        >>
        chunk c_construction:search_space_rules.rules.rhs:_2
            + io.construction_signal ** signal_tokens.st


4it [00:00, 24708.71it/s]
4it [00:00, 44858.87it/s]


event 0x0000 00:00:00.00 096 3 response_rules.rules.lhs.compile_weights
event 0x0000 00:00:00.00 096 4 response_rules.rules.rhs.compile_weights
event 0x0000 00:00:00.00 096 5 response_rules.rules.compile_weights
event 0x0000 00:00:00.00 096 6 blas.invoke
event 0x0000 00:00:00.00 096 7 search_space_rules.rules.lhs.compile_weights
event 0x0000 00:00:00.00 096 8 search_space_rules.rules.rhs.compile_weights
event 0x0000 00:00:00.00 096 9 search_space_rules.rules.compile_weights
event 0x0000 00:00:00.00 096 10 search_space_blas.invoke
event 0x0000 00:00:01.00 064 2 p1.start_construct_trial
Stimulus grid:  [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 2 0 0]
 [0 0 2 2 0 0]
 [4 4 4 0 0 0]]
Query brick 1:  4
Query brick 2:  2
Query relation:  1
event 0x0000 00:00:01.00 064 11 construction_input.send


...


2it [00:00, 20610.83it/s]
100%|██████████| 2916/2916 [00:00<00:00, 43987.97it/s]

event 0x0000 00:00:01.10 112 62 choice.select
event 0x0000 00:00:01.10 064 60 response_rules.rules.rhs.td.update



2it [00:00, 19418.07it/s]
100%|██████████| 2916/2916 [00:00<00:00, 49816.26it/s]

event 0x0000 00:00:01.10 112 65 choice.select
event 0x0000 00:00:01.10 064 63 p1.finish_response_trial


event 0x0000 00:00:01.10 064 66 p1.finish_response_trial
event 0x0000 00:00:01.10 064 67 p1.start_construct_trial


In [11]:
np.where(np.array([[0, 0, 1], [0, 1, 1], [0, 0, 0], [0,0, 0], [0, 0, 0], [0, 0,0]]) == 1)

(array([0, 1, 1]), array([2, 1, 2]))